In [5]:
from neo4j import GraphDatabase

In [6]:
URI = "neo4j+s://398500ef.databases.neo4j.io"
AUTH = ("neo4j", "l_1gCVeywnEgHdmLxf93BdT7kp01G30o5ZpTORaI-y4")

In [7]:
driver = GraphDatabase.driver(URI, auth=(AUTH))

In [3]:
def run_query(query, parameters=None):
    with driver.session() as session:
        results = session.run(query, parameters)
        return [dict(record) for record in results]

In [15]:
def create_user(name, age, location, interests):
    query = """
            CREATE (u:User {name: $name, age: $age, location: $location, interests: $interests})
            RETURN u
            """
    parameters = {"name": name, "age": age, "location":location, "interests":interests}
    result = run_query(query, parameters)
    print(result)

[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


In [17]:
def update_user(name, **kwargs):
    set = ",".join([f"u.{key} = ${key}" for key in kwargs])
    query = ("MATCH (u:User {name: $name}) "
            f"SET {set} "
            "RETURN u")
            
    parameters = {"name": name, **kwargs}
    result = run_query(query, parameters)
    print(result)

[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>}]


In [18]:
def create_friendship(user1, user2):
    query = """
            MATCH (u1:User {name: $user1}),
            (u2:User {name: $user2})
            CREATE (u1)-[:FRIENDS_WITH]->(u2), (u2)-[:FRIENDS_WITH]->(u1)
            RETURN u1, u2
            """
    parameters = {"user1": user1, "user2": user2}
    result = run_query(query, parameters)
    print(result)


[{'u1': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>, 'u2': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


In [53]:
def remove_friendship(user1, user2):
    query = """
            MATCH (u1:User {name: $user1})-[r:FRIENDS_WITH]-(u2:User {name: $user2})
            DELETE r
            """
    parameters = {"user1": user1, "user2": user2}
    result = run_query(query, parameters)
    print(result)

[]


In [50]:
def send_friend_request(user1, user2):
    query = """
            MATCH (u1:User {name: $user1}),
            (u2:User {name: $user2})
            CREATE (u1)-[:OUTGOING_REQUEST]->(u2), (u2)-[:INCOMING_REQUEST]->(u1)
            RETURN u1, u2
            """
    parameters = {"user1": user1, "user2": user2}
    result = run_query(query, parameters)
    print(result)

[{'u1': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>, 'u2': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:20' labels=frozenset({'User'}) properties={'name': 'Bob', 'location': 'California', 'interests': ['movies', 'reading'], 'age': 25}>}]


In [59]:
def respond_to_friend_request(user1, user2, accept):
    query = (
            "MATCH (u1:User {name: $from_user})-[r1:OUTGOING_REQUEST]->(u2:User {name: $to_user}) "
            "MATCH (u2)-[r2:INCOMING_REQUEST]->(u1) "
            "DELETE r1, r2 "
    )
    if accept:
        query += "CREATE (u1)-[:FRIENDS_WITH]->(u2), (u2)-[:FRIENDS_WITH]->(u1)"
    
    parameters = {"from_user": user1, "to_user": user2}
    result = run_query(query, parameters)
    print(result)

[]


In [61]:
def post(user, post):
    query = """
            MATCH (u:User {name: $user})
            CREATE (u)-[:POSTED]->(p:Post {content: $post})
            RETURN p.id
            """
    parameters = {"user": user, "post": post}
    result = run_query(query, parameters)
    print(result)

[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


In [77]:
def like_post(user, post_id):
    query = """
            MATCH (u:User {name: $user})
            MATCH (p:Post) WHERE elementId(p) = $post
            CREATE (u)-[:LIKED]->(p)
            RETURN p
            """
    parameters = {"user": user, "post": post_id}
    result = run_query(query, parameters)
    print(result)

[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


In [1]:
def create_group(name):
    query = """
            CREATE (g:Group {name: $name})
            RETURN g
            """
    parameters = {"name": name}
    result = run_query(query, parameters)
    print(result)

[{'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:41' labels=frozenset({'Group'}) properties={'name': 'admin'}>}]


In [11]:
def join_group(group, user):
    query = """
            MATCH (g:Group {name: $name})
            MATCH (u:User {name: $user})
            CREATE (u)-[:MEMBER_OF]->(g)
            RETURN u, g
            """
    parameters = {"name": group, "user": user}
    result = run_query(query, parameters)
    print(result)

[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>, 'g': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:41' labels=frozenset({'Group'}) properties={'name': 'admin'}>}]


In [12]:
def comment_on_post(user, post, comment):
    query = """
            MATCH (u:User {name: $user})
            MATCH (p:Post) WHERE elementId(p) = $post
            CREATE (u) -[:COMMENTED{comment: $comment}]-> (p)
            RETURN p
            """
    parameters = {"user": user, "post": post, "comment": comment }
    result = run_query(query, parameters)
    print(result)

[{'p': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:22' labels=frozenset({'Post'}) properties={'content': 'try again'}>}]


In [22]:
def friend_recommendation(user):
    query = """
            MATCH (u:User{name:$user})-[:FRIENDS_WITH] -> (u2) <-[:FRIENDS_WITH]- (ru)
            WHERE NOT (u)-[:FRIENDS_WITH] -> (ru) and u <> ru
            RETURN ru
            """
    parameters = {"user": user}
    result = run_query(query, parameters)
    print(result)

[{'ru': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:42' labels=frozenset({'User'}) properties={'name': 'Charlie', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 22}>}]


[{'u': <Node element_id='4:73dec725-ccdf-437b-bfd5-623161b3662c:19' labels=frozenset({'User'}) properties={'name': 'Alice', 'location': 'New York', 'interests': ['Music', 'Running'], 'age': 32}>}]
